<a href="https://colab.research.google.com/github/algomaster99/meta-btp/blob/master/work_hardening_rate.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

### Dependencies

#### Third party

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

%cd '/content/drive/MyDrive/Colab Notebooks'

from matplotlib import pyplot as plt
import numpy as np

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
/content/drive/MyDrive/Colab Notebooks


#### Personal notebooks

In [ ]:
!pip install import-ipynb
import import_ipynb
from utilities import differentiate
# import engineering_stress_strain_to_true_stress_strain
from engineering_stress_strain_to_true_stress_strain import (
  true_plastic_strain,
  true_stress,
  yield_strength,
)

ImportError: ignored

### Notations
- $\sigma \rightarrow$ True Stress
- $\epsilon \rightarrow$ True Strain
- $\epsilon_{pl} \rightarrow$ True Plastic Strain
- $\sigma_{ys} \rightarrow$ Yield Strength
- $\sigma_{uts} \rightarrow$ Ultimate Tensile Strength
- $n \rightarrow$ Work hardening Exponent
- $E \rightarrow$ Young's Modulus
- $K \rightarrow$ Strength Coefficient
- $e \rightarrow$ Engineering Strain
- $s \rightarrow$ Engineering Stress
- $\theta \rightarrow$ Work hardening rate
- $\sigma - \sigma_{ys} \rightarrow$ Flow Stress

## Calculation of constants in Hollomon equation

$$ \sigma = K \cdot \epsilon^n $$
$$ ln(\sigma) = ln(K) + n \cdot ln(\epsilon)$$

In [ ]:
log_true_plastic_strain = np.log(true_plastic_strain)
_ = plt.plot(log_true_plastic_strain, log_true_stress)
_ = plt.xlabel('$ln(\epsilon_{pl})$')
_ = plt.ylabel('$ln(\sigma)$')

AttributeError: ignored

In [ ]:
# Considering true plastic strain instead of true strain for calculating work
# hardening exponent and strength coefficient.

reshaped_log_true_plastic_strain = log_true_plastic_strain.reshape((-1,1))
model = LinearRegression().fit(reshaped_log_true_plastic_strain, log_true_stress)
coef = model.score(reshaped_log_true_plastic_strain, log_true_stress)
work_hardening_exponent = model.coef_
logk = model.intercept_
strength_coefficient = np.exp(logk)
print ('Work hardening exponent:', work_hardening_exponent)
print('coefficient of determination:', coef)
print('Strength coefficient:', strength_coefficient)
Latex(r'$\sigma = %f \times \epsilon_{pl} ^ {%f}$' % (strength_coefficient, work_hardening_exponent[0]))

## Comparison of hollomon equation and experiemental data

In [ ]:
theoretical_stress = [strength_coefficient*(true_plastic_strain[i]**work_hardening_exponent[0]) for i in range(len(true_plastic_strain))]
plt.plot(true_plastic_strain, theoretical_stress, label='Theoretical Stress')
plt.plot(true_plastic_strain, true_stress, label='Experimental Stress')
plt.legend(loc='upper left')

## Work Hardening Rate plot

In [ ]:
 _, _, d_sigma_epsilon, _ = differentiate(true_plastic_strain, true_stress)
# print(len(d_sigma_epsilon))
# plt.scatter(true_plastic_strain[:len(true_plastic_strain)-1], d_sigma_epsilon)

def func(strain, K, n):
  return K*(strain**n)

popt, pcov = optimize.curve_fit(func, true_plastic_strain[:len(true_plastic_strain)-1], d_sigma_epsilon, p0=[np.exp(logk), work_hardening_exponent[0]])
print(popt)

plt.plot(true_plastic_strain[:len(true_plastic_strain)-1], func(true_plastic_strain[:len(true_plastic_strain)-1], *popt), 'r-')